In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
# from utils import read_wav, extract_feats, read_dataset, batch, decode
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from IPython.core.display import HTML
from scipy.signal import spectrogram

from keras.layers import LSTM, Dense, Convolution1D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed, Bidirectional

%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
import os
import numpy as np
import pickle
import scipy.io.wavfile as wav
from itertools import izip_longest as zip_longest

In [ ]:
def decode(d, mapping):
    """Decode."""
    shape = d.dense_shape
    batch_size = shape[0]
    ans = np.zeros(shape=shape, dtype=int)
    seq_lengths = np.zeros(shape=(batch_size, ), dtype=np.int)
    for ind, val in zip(d.indices, d.values):
        ans[ind[0], ind[1]] = val
        seq_lengths[ind[0]] = max(seq_lengths[ind[0]], ind[1] + 1)
    ret = []
    for i in range(batch_size):
        ret.append("".join(map(lambda s: mapping[s], ans[i, :seq_lengths[i]])))
    return ret
def list_2d_to_sparse(list_of_lists):
    """Convert python list of lists to a [tf.SparseTensorValue](https://www.tensorflow.org/api_docs/python/tf/SparseTensorValue).

    Args:
        list_of_lists: list of lists to convert.

    Returns:
        tf.SparseTensorValue which is a namedtuple (indices, values, shape) where:

            * indices is a 2-d numpy array with shape (sum_all, 2) where sum_all is a
            sum over i of len(l[i])

            * values is a 1-d numpy array with shape (sum_all, )

            * shape = np.array([len(l), max_all]) where max_all is a max over i of
            len(l[i])

        Also, the following is true: for all i values[i] ==
        list_of_lists[indices[i][0]][indices[i][1]]

    """
    indices, values = [], []
    for i, sublist in enumerate(list_of_lists):
        for j, value in enumerate(sublist):
            indices.append([i, j])
            values.append(value)
    dense_shape = [len(list_of_lists), max(map(len, list_of_lists))]
    return tf.SparseTensorValue(indices=np.array(indices),
                                values=np.array(values),
                                dense_shape=np.array(dense_shape))


In [3]:
import time

In [4]:
vocabulary = { 'а': 1,
               'б': 2,
               'в': 3,
               'г': 4,
               'д': 5,
               'е': 6,
               'ё': 7,
               'ж': 8,
               'з': 9,
               'и': 10,
               'й': 11,
               'к': 12,
               'л': 13,
               'м': 14,
               'н': 15,
               'о': 16,
               'п': 17,
               'р': 18,
               'с': 19,
               'т': 20,
               'у': 21,
               'ф': 22,
               'х': 23,
               'ц': 24,
               'ч': 25,
               'ш': 26,
               'щ': 27,
               'ъ': 28,
               'ы': 29,
               'ь': 30,
               'э': 31,
               'ю': 32,
               'я': 33}

inv_mapping = dict(zip(vocabulary.values(), vocabulary.keys()))
inv_mapping[34]='<пробел>'

In [5]:
X = pickle.load( open( "X_pncc.pkl", "rb" ) )

In [6]:
y = pickle.load( open( "y.pkl", "rb" ) )


In [7]:
def decode2(stroka,inv_mapping,session):
    dense_decoded = tf.sparse_tensor_to_dense(stroka, default_value=-1).eval(session=session)

    seq = [s for s in dense_decoded[0] if s != -1]
    ret = []
    for i in range(len(seq)):
         ret.append("".join( inv_mapping[seq[i]]))
    for i in range(len(ret)):
        print(str(ret[i])),      

In [8]:
def decode1(stroka,inv_mapping):
    ret = []
    for i in range(len(stroka)):
         ret.append("".join( inv_mapping.get(int(stroka[i]),34)))
    for i in range(len(ret)):
        print(str(ret[i])),
    print('')

In [9]:
from utils import  list_2d_to_sparse
from itertools import izip_longest as zip_longest

In [10]:
def batch(X_train, y_train, batch_size):
    num_features = X_train[0].shape[1]
    n = len(X_train)
    perm = np.random.permutation(n)
    for batch_ind in np.resize(perm, (n // batch_size, batch_size)):
        X_batch, y_batch = [X_train[i] for i in batch_ind], [y_train[i] for i in batch_ind]
        sequence_lengths = list(map(len, X_batch))
        X_batch_padded = np.array(list(zip_longest(*X_batch, fillvalue=np.zeros(num_features)))).transpose([1, 0, 2])
        yield X_batch_padded, sequence_lengths, list_2d_to_sparse(y_batch), y_batch

In [43]:
graph = tf.Graph()
with graph.as_default():
    input_X = tf.placeholder(tf.float32, shape=[None, None, 13],name="input_X")
    labels = tf.sparse_placeholder(tf.int32)
    seq_lens = tf.placeholder(tf.int32, shape=[None],name="seq_lens")


    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2), input_shape=(None, 13)))
   
    model.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2)))
  
    model.add(TimeDistributed(Dense(len(inv_mapping) + 2)))
    
    final_seq_lens = seq_lens

    logits = model(input_X)
    logits = tf.transpose(logits, [1, 0, 2])

    ctc_loss = tf.reduce_mean(tf.nn.ctc_loss(labels, logits, final_seq_lens,ignore_longer_outputs_than_inputs=True))
    # ctc_greedy_decoder? merge_repeated=True
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, final_seq_lens)
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), labels))

    train_op = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(ctc_loss)

In [12]:
def decode_single(session, test_input):
    z=np.zeros((30,13))
    zz=np.vstack((test_input,z))

    val_feed = {
        input_X:  np.asarray([zz]),
        seq_lens: np.asarray([len(test_input)])
    }

    # Decoding
    d = session.run(decoded[0], feed_dict=val_feed)
    dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=session)

    seq = [s for s in dense_decoded[0] if s != -1]
    print('Decoded:\t%s' % (decode2(d, inv_mapping,session)))
    print(seq)

In [37]:
y_train1=[]
X_train1=[]
for i in range(len(y)):
    if len(y[i]) <75:
        y_train1.append(y[i])
        X_train1.append(X[i])
len(y_train1)

5817

In [43]:
num_epochs=200
epoch_save_step=20


In [44]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint3")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_train1, y_train1, 100):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,
                
                seq_lens: seq_lens_batch
            }
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint3/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
        print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret=decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)

(0, 121.66106, 0.99999994)
          Mon Sep 18 23:52:26 2017
<пробел> р а з б у д и <пробел> в <пробел> ш е с т ь <пробел> у т р а <пробел> 
(1, 119.36066, 0.99999994)
          Mon Sep 18 23:55:48 2017
<пробел> н е <пробел> з н а ю <пробел> ч т о <пробел> о н <пробел> п о н и м а е т <пробел> п о д <пробел> э т и м <пробел> 
(2, 117.67046, 1.0)
          Mon Sep 18 23:59:08 2017
<пробел> п о л о ж и л <пробел> в <пробел> ч е м о д а н <пробел> и <пробел> з а б ы л <пробел> 
(3, 107.12263, 1.0)
          Tue Sep 19 00:02:36 2017
<пробел> я <пробел> д о л ж н а <пробел> б ы т ь <пробел> г д е н и б у д ь <пробел> в <пробел> д р у г о м <пробел> м е с т е <пробел> 
(4, 106.62258, 0.99616349)
    <пробел>      Tue Sep 19 00:06:03 2017
<пробел> т ы <пробел> с и д е л <пробел> у <пробел> о т ц а <пробел> н а <пробел> р у к а х <пробел> и <пробел> о т е ц <пробел> г о в о р и л <пробел> 
(5, 93.227638, 0.83988762)
<пробел>котепооа<пробел> <пробел>д<пробел>от<пробел>а<пробел> <пробел><пробел

(15, 55.527107, 0.35176826)
<пробел>осену<пробел>етие<пробел>наат<пробел>меущ<пробел>земны<пробел>оаклесто<пробел> <пробел>мара<пробел> <пробел>потуши<пробел>свет<пробел> <пробел>мара<пробел> <пробел>какая<пробел>погода<пробел>по<пробел>идерь<пробел>вудеть<пробел> <пробел>нодое<пробел>мар<пробел>тргу<пробел>ра<пробел>уму<пробел>ка<пробел>пановмат<пробел>стал<пробел>но<пробел>додать<пробел> <пробел>обос<пробел>отогея<пробел>и<пробел>би<пробел><пробел>вечрны<пробел>дёатожно<пробел>уемндет<пробел> <пробел>лицо<пробел>го<пробел>вэт<пробел>минуту<пробел>казлало<пробел>ь<пробел>толёны<пробел>лоб<пробел>сморлся<пробел> о<пробел>боеших<пробел>пудлите<пробел>ни<пробел> <пробел>нзьмар<пробел>чьто<пробел>это<пробел>посклнн<пробел>нач<пробел> Tue Sep 19 00:43:26 2017
<пробел> о с е н н и й <пробел> в е т е р <пробел> н а д <пробел> к а т е й <пробел> л е ж а щ е й <пробел> в <пробел> з е м л е <пробел> п о д <пробел> к р е с т о м <пробел> 
(16, 47.116058, 0.31539786)
<пробел>казалос<пробел>исчно<

(25, 27.390804, 0.21212552)
<пробел>ны<пробел>незнаноатчем<пробел><пробел>оварить<пробел> <пробел>мара<пробел> <пробел>мара<пробел> <пробел>лос<пробел>кинуля<пробел>естаа<пробел>попали<пробел>аесе<пробел>иванови<пробел>притягивымся<пробел>падые<пробел> <пробел>мара<пробел> <пробел>какой<пробел>сегодня<пробел>день<пробел> <пробел>сначала<пробел>думули<пробел>чтоэтосдыбуль<пробел>магиеных<пробел>поля<пробел>о<пробел>зели<пробел> <пробел>быложе<пробел>ни<пробел>гы<пробел>гожне<пробел>стал<пробел>праи<пробел> <пробел>мара<пробел> <пробел>потуши<пробел>свет<пробел> Tue Sep 19 01:17:16 2017
<пробел> м ы <пробел> н е <пробел> з н а е м <пробел> о <пробел> ч е м <пробел> г о в о р и т ь <пробел> 
(26, 33.158241, 0.23238742)
<пробел>семь<пробел>ле<пробел>меня<пробел>ни<пробел>пуля<пробел>ни<пробел>шток<пробел>не<пробел>могли<пробел>истрбить<пробел> <пробел>мара<пробел> <пробел>качта<пробел>ственали<пробел> <пробел>на<пробел>ш<пробел>лод<пробел>озадахномпятор<пробел>заь<пробел>очал<пробел>прлоы<

(34, 26.176769, 0.17769477)
<пробел>сначала<пробел>думали<пробел>что<пробел>это<пробел>следы<пробел>бурь<пробел>понитых<пробел>поляхземи<пробел> <пробел><пробел>лаких<пробел>сенах<пробел>узкие<пробел>нак<пробел>щщели<пробел>лпоиеврсти<пробел>окны<пробел> <пробел>между<пробел>зубами<пробел>уних<пробел>была<пробел>трава<пробел>они<пробел>пались<пробел>там<пробел>где<пробел>теперь<пробел>льды<пробел> <пробел>мара<пробел> <пробел>мы<пробел>втянули<пробел>го<пробел>илон<пробел>упал<пробел>разбил<пробел>зенуюкоару<пробел>отклонил<пробел>полньс<пробел> <пробел>наа<пробел>боже<пробел>мо<пробел>ки<пробел>могли<пробел>быь<пробел>дела<пробел>важнее<пробел>катюшёной<пробел>люторни<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченелый<пробел>тру<пробел>влети<пробел>в<пробел>её<пробел>огненый<пробел>окены<пробел> <пробел>осачала<пробел>думи<пробел>что<пробел>эо<пробел>следы<пробел>борь<пробел>манитных<пробел>полях<пробел>емли<пробел> <пробел>раздался<пробел>жевез<пробел>ны<проб

(42, 17.307373, 0.11748143)
<пробел>не<пробел>знаю<пробел>чтон<пробел>по<пробел>нимаоет<пробел>одтям<пробел> <пробел>пдизадс<пробел>себе<пробел>что<пробел>ждал<пробел>недаки<пробел>по<пробел>снтитемю<пробел> <пробел>свет<пробел>проникал<пробел>внё<пробел>сквоь<пробел>забраные<пробел>стикклом<пробел>тверрстия<пробел>сводчатого<пробел>купола<пробел> <пробел>мара<пробел> <пробел>серезтысят<пробел>лет<пробел>мой<пробел>окченелый<пробел>тру<пробел>влетит<пробел>в<пробел>е<пробел>огнинн<пробел>океноны<пробел> <пробел>я<пробел>понима<пробел>ввеч<пробел>не<пробел>таке<пробел>какини<пробел>гони<пробел>касза<пробел>лис<пробел>бата<пробел> <пробел>потуши<пробел>све<пробел> <пробел>что<пробел>у<пробел>этой<пробел>барышни<пробел>было<пробел>в<пробел>голове<пробел>не<пробел>поймудо<пробел>сих<пробел>пор<пробел> <пробел>мара<пробел> <пробел>потуши<пробел>свет<пробел> Tue Sep 19 02:14:49 2017
<пробел> н е <пробел> з н а ю <пробел> ч т о <пробел> о н <пробел> п о н и м а е т <пробел> п о д <пробел> э т

...

(97, 3.813709, 0.026147082)
<пробел>тринадцать<пробел>четырнадцать<пробел>пятнадцать<пробел>шестнадцать<пробел>семнадцать<пробел> <пробел>господа<пробел>офицеры<пробел>мол<пробел>сами<пробел>знают<пробел>который<пробел>том<пробел>должны<пробел>читать<пробел> <пробел>в<пробел>деревню<пробел>ехать<пробел>отец<пробел>с<пробел>матерью<пробел>померли<пробел>бртья<пробел>убиты<пробел>земля<пробел>заброшена<пробел> <пробел>ведь<пробел>ад<пробел>и<пробел>есть<пробел>моё<пробел>безнадёжно<пробел>одиночство<пробел>распростёртое<пробел>в<пробел>вечной<пробел>тьме<пробел> <пробел>в<пробел>гладких<пробел>стенах<пробел>узкие<пробел>как<пробел>щели<пробел>глубокие<пробел>отверстия<пробел>окон<пробел> <пробел>потуши<пробел>свет<пробел> <пробел>ему<пробел>очень<пробел>хотелось<пробел>пойити<пробел>туда<пробел> <пробел>осенний<пробел>ветер<пробел>над<пробел>катей<пробел>лежащей<пробел>в<пробел>земле<пробел>под<пробел>крестом<пробел> <пробел>мара<пробел> <пробел>я<пробел>никогна<пробел>не<пробел>хочу<про

(105, 1.207345, 0.0049041002)
<пробел>со<пробел>спокойным<пробел>мужеством<пробел>скайлс<пробел>ожидал<пробел>всего<пробел>в<пробел>этом<пробел>безумном<пробел>городе<пробел> <пробел>мара<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченелый<пробел>труп<пробел>влетит<пробел>в<пробел>её<пробел>огненные<пробел>океаны<пробел> <пробел>глазаленивые<пробел>серокарие<пробел>и<пробел>так<пробел>же<пробел>как<пробел>у<пробел>той<пробел>женщины<пробел>с<пробел>искоркой<пробел> <пробел>учился<пробел>на<пробел>медные<пробел>деньги<пробел>с<пробел>двенадцати<пробел>лет<пробел>сам<пробел>их<пробел>зарабатываю<пробел> <пробел>сколько<пробел>время<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченелый<пробел>труп<пробел>влетит<пробел>в<пробел>её<пробел>огненные<пробел>океаны<пробел> <пробел>раскажи<пробел>что<пробел>нибудь<пробел>смешное<пробел> <пробел>мара<пробел> <пробел>глаза<пробел>ленивые<пробел>серокарие<пробел>и<пробел>так<пробел>же<пробел>как<пробел>

(113, 6.3301964, 0.047788642)
<пробел>какой<пробел>сегодня<пробел>день<пробел> <пробел>так<пробел>же<пробел>каки<пробел>говорить<пробел>сней<пробел>об<пробел>этом<пробел> <пробел>мара<пробел> <пробел>между<пробел>зубами<пробел>у<пробел>них<пробел>была<пробел>трава<пробел>они<пробел>паслись<пробел>там<пробел>где<пробел>теперь<пробел>льды<пробел> <пробел>вп<пробел>асоцатце<пробел>от<пробел>дюи<пробел>я<пробел>перешёл<пробел>к<пробел>ийеомдюфт<пробел> <пробел>приглуши<пробел>свет<пробел> <пробел>мара<пробел> <пробел>ах<пробел>боже<пробел>мой<пробел>какие<пробел>могли<пробел>быть<пробел>дела<пробел>важнее<пробел>катюшнойй<пробел>любви<пробел> <пробел>был<пробел>ли<пробел>он<пробел>там<пробел>вообще<пробел> <пробел>мара<пробел> Tue Sep 19 06:14:53 2017
<пробел> к а к о й <пробел> с е г о д н я <пробел> д е н ь <пробел> 
(114, 4.3194304, 0.027333146)
<пробел>мышь<пробел>понемногу<пробел>ожила<пробел>подняла<пробел>нос<пробел>стал<пробел>шевелить<пробел>усами<пробел>умылась<пробел> <пробел>ех

(121, 0.64049625, 0.001942008)
<пробел>что<пробел>он<пробел>взял<пробел>после<пробел>смерти<пробел>зубра<пробел>на<пробел>память<пробел>об<пробел>учителе<пробел> <пробел>лицо<пробел>его<пробел>в<пробел>эту<пробел>минуту<пробел>казалось<пробел>утомлённым<пробел>лоб<пробел>сморщился<пробел> <пробел>мы<пробел>втянули<пробел>его<пробел>и<пробел>он<пробел>упал<пробел>разбил<пробел>земную<пробел>кору<пробел>отклонил<пробел>полюсы<пробел> <пробел>какой<пробел>сегодня<пробел>день<пробел> <пробел>какой<пробел>сегодня<пробел>день<пробел> <пробел>включи<пробел>свет<пробел> <пробел>сначала<пробел>думали<пробел>что<пробел>это<пробел>следы<пробел>бурь<пробел>в<пробел>магнитных<пробел>полях<пробел>земли<пробел> <пробел>надоело<пробел>мало<пробел>толку<пробел>да<пробел>уж<пробел>и<пробел>мужикам<пробел>махновщина<пробел>эта<пробел>стала<пробел>надоедать<пробел> <пробел>мара<пробел> <пробел>поставь<пробел>будильник<пробел>на<пробел>восемь<пробел>тридцать<пробел> Tue Sep 19 06:41:58 2017
<пробел> ч т о 

KeyboardInterrupt: 

In [20]:
k=[]
y_train1=[]
X_train1=[]
for i in range(len(y)):
    if (len(y[i]) <181) and (len(X[i])<1745):
        y_train1.append(y[i])
        X_train1.append(X[i])
len(y_train1)

10487

In [44]:
num_epochs=300
epoch_save_step=5

In [30]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint3")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_train1, y_train1, 50):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,
                
                seq_lens: seq_lens_batch
            }
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint3/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
        print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret=decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)

[i] LOADING checkpoint checkpoint3/ctc.ckpt-120
INFO:tensorflow:Restoring parameters from checkpoint3/ctc.ckpt-120
[i] start from epoch 121
(121, 105.5779, 0.42037344)
<пробел>ронинез<пробел>власко<пробел>колол<пробел>премо<пробел>послерли<пробел>тлучолс<пробел> <пробел>одимо<пробел>час<пробел>расва<пробел><пробел>чте<пробел>яст<пробел>с<пробел>сеь<пробел> <пробел>вакалаво<пробел>вогруть<пробел>точавакрувки<пробел>выдаша<пробел>асье<пробел>кро<пробел>воз<пробел>дубу<пробел>понуквасу<пробел><пробел> <пробел>веэтой<пробел>патерен<пробел>пзнучалвсево<пробел>дажта<пробел> <пробел>нанижнисту<пробел>пени<пробел>онастонавилос<пробел>было<пробел>прянаспоять<пробел>пнуесонично<пробел>восбета<пробел>ходящевосквось<пробел>уско<пробел>окно<пробел> <пробел>едеть<пробел>пбоьсавемона<пробел> <пробел>нудрияэтоовтаракожн<пробел>быстаганавыица<пробел>походелес<пробел>апракнепледенидеженитесаонныбат<пробел>дяшкире<пробел>пола<пробел>сенел<пробел>нетис<пробел>оты<пробел>пола<пробел>подувкииар<пробел>нетаз

(127, 55.687302, 0.2479203)
<пробел>обоз<пробел>отобьён<пробел>и<пробел>к<пробел>лееру<пробел>мы<пробел>вёст<пробел>уж<пробел>за<пробел>осемьдесят<пробел> <пробел>эжку<пробел>збоиу<пробел>небылокд<пробел>рава<пробел>никтаслеь<пробел>стко<пробел>кет<пробел>ибх<пробел>тык<пробел> <пробел>привет<пробел>ка<пробел>дела<пробел>це<пробел> <пробел>обоз<пробел>отобь<пробел>и<пробел>клечеруну<пробел>вёрчтужа<пробел>залосеьмисьт<пробел> <пробел>разбуди<пробел>шисть<пробел>утра<пробел> <пробел>водин<пробел>и<пробел>лазков<пробел>входил<пробел>прямой<пробел>ослипительны<пробел>гуч<пробел>цснсцано<пробел> <пробел>электричествобыла<пробел>потушна<пробел>и<пробел>бледный<пробел>свет<пробел>проникал<пробел>сквозьтёкла<пробел>глазково<пробел> <пробел>на<пробел>глаз<пробел>скайльсо<пробел>пределило<пробел>ношсо<пробел>цсис<пробел>образна<пробел>прме<пробел>был<пробел>не<пробел>мени<пробел>восмис<пробел>половини<пробел>метроввсоты<пробел>и<пробел>шести<пробел>метров<пробел>ппречнке<пробел> <пробел>зно<про

(133, 32.88311, 0.14803064)
<пробел>мара<пробел> <пробел>нодожнее<пробел>отвить<пробел>ко<пробел>штов<пробел> <пробел>лось<пробел>жадно<пробел>вглядывалсяв<пробел>эту<пробел>сетьвлини<пробел>вот<пробел>они<пробел>сводящие<пробел>с<пробел>умо<пробел>астраномо<пробел>постоянно<пробел>меняющис<пробел>геометрически<пробел>правильны<пробел>непостигаемые<пробел>каналы<пробел>марса<пробел> <пробел>идти<пробел>было<пробел>неоычайно<пробел>легко<пробел>хотя<пробел>ноги<пробел>и<пробел>вязли<пробел>пощиколотку<пробел><пробел>рассыпающей<пробел>почеве<пробел> <пробел>иди<пробел>было<пробел>необычано<пробел>легко<пробел>хотя<пробел>ноги<пробел>и<пробел>язли<пробел>по<пробел>щиколотку<пробел>расыпающейспоче<пробел> <пробел>какой<пробел>сегодня<пробел>день<пробел> <пробел>некто<пробел>не<пробел>мос<пробел>нечегосьделат<пробел>протьссмене<пробел> <пробел>лось<пробел><пробел>кхратце<пробел>расказал<пробел>ме<пробел>о<пробел>буслових<пробел>перелёто<пробел>предепреело<пробел>возможномриспе<пробел> <про

(138, 47.135983, 0.15369391)
<пробел>оедю<пробел>быловина<пробел>омистаравнино<пробел>тогоже<пробел>пельсиноо<пробел>боле<пробел>ысклосме<пробел> <пробел>не<пробел>догняшей<пробел>кущдевьи<пробел>убло<пробел>горелому<пробел>бурому<пробел>ра<пробел>ере<пробел>розипавшиеся<пробел>нешигодкело<пробел>крудикил<пробел> <пробел>вывмеждвёздном<пробел>пространстве<пробел>носят<пробел>ся<пробел>сколки<пробел>планет<пробел>нерождённых<пробел>или<пробел>погиибших<пробел>миров<пробел> <пробел>мышь<пробел>понемногу<пробел>ожива<пробел>поняла<пробел>нос<пробел>стала<пробел>шжевелить<пробел>усами<пробел>умылась<пробел> <пробел>надоело<пробел>мало<пробел>толку<пробел>ве<пробел>гужн<пробел>муж<пробел>дам<пробел>махднощиниснааддам<пробел> <пробел>как<пробел>шли<пробел>мынынчев<пробел>носту<пробел>пленна<пробел> <пробел>надину<пробел>пальто<пробел>изашагал<пробел>через<пробел>пустырь<пробел>на<пробел>больурю<пробел>моетную<пробел>лулицу<пробел>ксее<пробел>но<пробел>квартиу<пробел>гиде<пробел>порлугоада<пр

(144, 28.555777, 0.12016701)
<пробел>онии<пробел>те<пробел>же<пробел>семена<пробел>оседает<пробел>на<пробел>марс<пробел>и<пробел>на<пробел>землю<пробел>на<пробел>взе<пробел>мириады<пробел>остывоиуи<пробел>звёзд<пробел> <пробел>он<пробел>цолот<пробел>евщёкив<пробел>вобо<пробел>в<пробел>закратые<пробел>веки<пробел> <пробел>ривычной<пробел>хвако<пробел>растягнул<пробел>кобуру<пробел>выдащел<пробел>реворевер<пробел>э<пробел>закриалокто<пробел>там<пробел>у<пробел>аппарат<пробел>таош<пробел>эда<пробел> <пробел>шли<пробел>с<пробел>намирежной<пробел>бежали<пробел>из<пробел>переулков<пробел>бубнили<пробел>бивались<пробел>а<пробел>кучки<пробел>лежали<пробел>на<пробел>чеахлый<пробел>прове<пробел>подлядывали<пробел>на<пробел>низкое<пробел>солнце<пробел>пустишее<пробел>сквозь<пробел>облака<пробел>широкие<пробел>лучи<пробел> <пробел>идти<пробел>было<пробел>небычаено<пробел>легко<пробел>хотя<пробел>ноги<пробел>и<пробел>вязды<пробел>пощиклотку<пробел>в<пробел>рассыпающеи<пробел>почве<пробел> <пробел>о

...

(253, 6.1670837, 0.028477669)
<пробел>я<пробел>вам<пробел>на<пробел>марсе<пробел>пригожусь<пробел>ну<пробел>очень<пробел>рад<пробел>казал<пробел>лось<пробел>подавая<пробел>ему<пробел>рук<пробел>до<пробел>завтра<пробел> <пробел>несколько<пробел>линий<пробел>точек<пробел>и<пробел>полукружий<пробел>разбросано<пробел>к<пробел>западу<пробел>и<пробел>востоку<пробел>от<пробел>этой<пробел>икваториальной<пробел>группы<пробел> <пробел>вообщето<пробел>беспорно<пробел>чту<пробел>на<пробел>небе<пробел>южного<пробел>полушарея<пробел>неподалёку<пробел>от<пробел>созвездия<пробел>большого<пробел>пса<пробел>уже<пробел>за<пробел>сорок<пробел>лет<пробел>до<пробел>того<пробел>было<пробел>открыто<пробел>созвездие<пробел>голубя<пробел> <пробел>в<пробел>междузвёздном<пробел>пространстве<пробел>носятся<пробел>осколки<пробел>планет<пробел>нерождённых<пробел>или<пробел>погибших<пробел>миров<пробел> <пробел>лицо<пробел>его<пробел>в<пробел>эту<пробел>мину<пробел>ту<пробел>казалось<пробел>утомлённым<пробел>лоб<проб

(258, 4.7218733, 0.024068594)
<пробел>этим<пробел>разковори<пробел>кончился<пробел> <пробел>скайльс<пробел>тоже<пробел>теперь<пробел>прлищурясь<пробел>оглянул<пробел>солдата<пробел>оскпызхнул<пробел>гневно<пробел>и<пробел>пошёл<пробел>по<пробел>направлиник<пробел>неве<пробел>шагал<пробел>уверенно<пробел>и<пробел>широко<пробел> <пробел>он<пробел>сел<пробел>к<пробел>столу<пробел>началупить<пробел>варёную<пробел>картошку<пробел>разломил<пробел>окуну<пробел>в<пробел>соль<пробел>на<пробел>завтра<пробел>приготовь<пробел>чистое<пробел>две<пробел>смены<пробел>рубашки<пробел>подштаники<пробел>подвёртте<пробел> <пробел>в<пробел>молчании<пробел>теперь<пробел>тысячи<пробел>людей<пробел>глядели<пробел>на<пробел>квадратные<пробел>освещённые<пробел>окна<пробел>сарая<пробел> <пробел>мара<пробел> <пробел>маленький<пробел>воротник<пробел>как<пробел>на<пробел>военной<пробел>форме<пробел> <пробел>лось<пробел>показал<пробел>гусеву<пробел>механизм<пробел>движения<пробел>и<пробел>важнешие<пробел>приборы<проб

(263, 5.9688354, 0.025849679)
<пробел>свет<пробел>проникал<пробел>в<пробел>неё<пробел>сквозь<пробел>забранные<пробел>стеклом<пробел>отверсти<пробел>содчатого<пробел>купола<пробел> <пробел>осенний<пробел>ветер<пробел>на<пробел>д<пробел>катей<пробел>лежащейв<пробел>земле<пробел>под<пробел>крестом<пробел> <пробел>через<пробел>тысячу<пробел>лет<пробел>мой<пробел>окоченилый<пробел>труп<пробел>влетит<пробел>в<пробел>её<пробел>огненные<пробел>океаны<пробел> <пробел>включи<пробел>свет<пробел> <пробел>над<пробел>головой<пробел>летела<пробел>не<пробел>многла<пробел>улететь<пробел>виесёлая<пробел>женщина<пробел>с<пробел>весёлыми<пробел>младенцами<пробел> <пробел>обоз<пробел>отобьём<пробел>и<пробел>к<пробел>вечеру<пробел>мыи<пробел>вёрст<пробел>уж<пробел>за<пробел>восемьдесят<пробел> <пробел>дло<пробел>сумек<пробел>переливал<пробел>ся<пробел>говор<пробел>шли<пробел>разные<пробел>разговоры<пробел>в<пробел>толпе<пробел>ожидающей<пробел>необыкновенного<пробел>события<пробел> <пробел>я<пробел>вам<проб

(268, 3.6000142, 0.017272593)
<пробел>мара<пробел> <пробел>его<пробел>ярко<пробел>освещённая<пробел>внутенность<пробел>из<пробел>стёганой<пробел>ромбами<пробел>жёлтой<пробел>кожи<пробел>была<пробел>видна<пробел>сквозь<пробел>круглое<пробел>отверстие<пробел>люка<пробел> <пробел>семь<пробел>лет<пробел>меня<пробел>ни<пробел>пуля<пробел>ни<пробел>штык<пробел>не<пробел>могли<пробел>истреьбить<пробел> <пробел>покалывало<пробел>грудь<пробел>стучала<пробел>кровь<пробел>в<пробел>виски<пробел>но<пробел>дышалось<пробел>легко<пробел>воздух<пробел>был<пробел>тонок<пробел>и<пробел>сух<пробел> <пробел>он<пробел>сел<пробел>к<пробел>столу<пробел>начал<пробел>лупить<пробел>варёную<пробел>картошку<пробел>разломил<пробел>окунул<пробел>в<пробел>соль<пробел>на<пробел>завтра<пробел>приготовь<пробел>чистое<пробел>две<пробел>смены<пробел>рубашки<пробел>подштаники<пробел>подвёртки<пробел> <пробел>мо<пробел>втянули<пробел>его<пробел>и<пробел>он<пробел>упал<пробел>разбил<пробел>земную<пробел>кору<пробел>отклонил<

KeyboardInterrupt: 

In [46]:
num_epochs=311
epoch_save_step=1

In [47]:
with tf.Session(graph=graph) as session:

    saver = tf.train.Saver(tf.global_variables())
    snapshot = "ctc"
    checkpoint = tf.train.latest_checkpoint(checkpoint_dir="checkpoint3")
    last_epoch = 0

    if checkpoint:
        print("[i] LOADING checkpoint " + checkpoint)
        try:
            saver.restore(session, checkpoint)
            last_epoch = int(checkpoint.split('-')[-1]) + 1
            print("[i] start from epoch %d" % last_epoch)
        except:
            print("[!] incompatible checkpoint, restarting from 0")
    else:
        # Initializate the weights and biases
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):
        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X, y, 10):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,
                
                seq_lens: seq_lens_batch
            }
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint3/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
        print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret=decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)

[i] LOADING checkpoint checkpoint3/ctc.ckpt-305
INFO:tensorflow:Restoring parameters from checkpoint3/ctc.ckpt-305
[i] start from epoch 306
[i] SAVING snapshot ctc
(306, 10.71156, 0.033768088)
<пробел>что<пробел>у<пробел>этой<пробел>барышни<пробел>было<пробел>в<пробел>голове<пробел>не<пробел>пойму<пробел>до<пробел>сих<пробел>пор<пробел> <пробел>мара<пробел> <пробел>в<пробел>это<пробел>время<пробел>перед<пробел>объявлением<пробел>остановился<пробел>рослый<пробел>широкоплечий<пробел>человек<пробел>без<пробел>шапки<пробел>по<пробел>одежде<пробел>солдат<пробел>в<пробел>рубахе<пробел>без<пробел>пояса<пробел>в<пробел>обмотках<пробел> <пробел>в<пробел>молчании<пробел>теперь<пробел>тысячи<пробел>людый<пробел>глядели<пробел>на<пробел>квадратные<пробел>освещённые<пробел>окна<пробел>сарая<пробел> <пробел>она<пробел>завела<пробел>прядь<пробел>волсяныхволус<пробел>за<пробел>ухо<пробел>подняла<пробел>с<пробел>тротуа<пробел>ра<пробел>корзику<пробел>с<пробел>зелини<пробел>и<пробел>пошла<пробел>чериз<п

[i] SAVING snapshot ctc
(309, 6.0868406, 0.040936153)
<пробел>е<пробел>назваю<пробел>три<пробел>зубра<пробел> <пробел>мара<пробел> <пробел>не<пробел>остася<пробел>ни<пробел>одной<пробел>душе<пробел> <пробел>лось<пробел>показал<пробел>гусеву<пробел>механизм<пробел>движения<пробел>и<пробел>важнейшие<пробел>приборы<пробел>гусев<пробел>оказался<пробел>ловким<пробел>и<пробел>смелавм<пробел>человеком<пробел> <пробел>сколько<пробел>время<пробел> <пробел>мышь<пробел>понемногу<пробел>ожила<пробел>подняла<пробел>нос<пробел>стала<пробел>шевелить<пробел>усами<пробел>умылась<пробел> <пробел>потом<пробел>взошёл<пробел>на<пробел>лесенку<пробел>телескопа<пробел>нашёл<пробел>искателе<пробел>марс<пробел>поднявшийся<пробел>ужевнад<пробел>петербургом<пробел>и<пробел>долго<пробел>глядел<пробел>она<пробел>небольшой<пробел>ясный<пробел>тёплый<пробел>шарик<пробел> <пробел>небольшая<пробел>дверца<пробел>в<пробел><пробел>нём<пробел>была<пробел>приотворена<пробел>на<пробел>пороге<пробел>сидел<пробел>на<пробел>ко